In [1]:
from infinipy.stateblock import StateBlock
import uuid

from infinipy.statement import Statement,CompositeStatement
from infinipy.transformer import Transformer,CompositeTransformer
from infinipy.affordance import Affordance
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Any, Union, Tuple


In [2]:
@dataclass
class Door(StateBlock):
    is_open: bool = False  # Attribute to indicate if the door is open
    is_locked: bool = False  # Attribute to indicate if the door is locked

    def __post_init__(self):
        super().__post_init__()  # Call the post-init of StateBlock
        # Additional initialization can be added here if needed


In [3]:
door = Door(
    id=str(uuid.uuid4()),
    owner_id="door_owner",  # Identifier of the door's owner or location
    name="Door",
    blocks_move=True,  # A closed door blocks movement
    blocks_los=False,  # A door doesn't generally block line of sight
    can_store=False,
    can_be_stored=False,
    can_act=False,
    can_move=False,
    can_be_moved=False,
    position=(0, 0, 0),  # Position of the door in the environment
    is_open=False,  # Custom attribute indicating if the door is open
    is_locked=True  # Custom attribute indicating if the door is locked
)


In [4]:
def is_open_condition(door: StateBlock):
    return door.is_open

def is_closed_condition(door: StateBlock):
    return not door.is_open

def is_locked_condition(door: StateBlock):
    return door.is_locked

def is_unlocked_condition(door: StateBlock):
    return not door.is_locked

statement_is_open = Statement(
    name="IsOpen",
    description="is open",
    condition=is_open_condition
)

statement_is_closed = Statement(
    name="IsClosed",
    description="is closed",
    condition=is_closed_condition
)

statement_is_locked = Statement(
    name="IsLocked",
    description="is locked",
    condition=is_locked_condition
)

statement_is_unlocked = Statement(
    name="IsUnlocked",
    description="is unlocked",
    condition=is_unlocked_condition
)
composite_statement_closed_locked = CompositeStatement([
    (statement_is_closed, "AND", "source"),
    (statement_is_locked, "AND", "source")
])

composite_statement_closed_or_locked = CompositeStatement([
    (statement_is_closed, "OR", "source"),
    (statement_is_locked, "OR", "source")
])

In [5]:
def open_door(door: Door):
    if not door.is_open:
        door.is_open = True


consequence_open_door = CompositeStatement([
    (statement_is_open, "AND", "source"),
    (statement_is_closed, "AND NOT", "source"),
])
prerequisite_open_door = CompositeStatement([
    (statement_is_closed, "AND", "source"),
    (statement_is_open, "AND NOT", "source"),
    (statement_is_unlocked, "AND", "source"),
    (statement_is_locked, "AND NOT", "source")
])


transformer_open_door = CompositeTransformer(
    transformers=[(
        Transformer(
        name="OpenDoor",
        transformation=open_door,
        consequences=consequence_open_door),"source")])

#composite transfo
def close_door(door: Door):
    if door.is_open:
        door.is_open = False


consequence_close_door = CompositeStatement([
    (statement_is_closed, "AND", "source"),
    (statement_is_open, "AND NOT", "source"),
])

prerequisite_close_door = CompositeStatement([
    (statement_is_open, "AND", "source"),
    (statement_is_closed, "AND NOT", "source"),
    (statement_is_unlocked, "AND", "source"),
    (statement_is_locked, "AND NOT", "source"),
])

transformer_close_door = CompositeTransformer(
    transformers=[(Transformer(
    name="CloseDoor",
    transformation=close_door,
    consequences=consequence_close_door
), "source")])

def lock_door(door: Door):
    if not door.is_locked:
        door.is_locked = True



prerequisite_lock_door = CompositeStatement([
    (statement_is_unlocked, "AND", "source"),
    (statement_is_locked, "AND NOT", "source"),

])

consequence_lock_door = CompositeStatement([
    (statement_is_locked, "AND", "source"),
    (statement_is_unlocked, "AND NOT", "source"),
])

transformer_lock_door = CompositeTransformer(
    transformers=[(Transformer(
    name="LockDoor",
    transformation=lock_door,
    consequences=consequence_lock_door
), "source")])


def unlock_door(door: Door):
    if door.is_locked:
        door.is_locked = False


prerequisite_unlock_door = CompositeStatement([
    (statement_is_locked, "AND", "source"),
    (statement_is_unlocked, "AND NOT", "source"),
    
])
consequence_unlock_door = CompositeStatement([
    (statement_is_unlocked, "AND", "source"),
    (statement_is_locked, "AND NOT", "source"),
])

transformer_unlock_door = CompositeTransformer(
    transformers=[(Transformer(
    name="UnlockDoor",
    transformation=unlock_door,
    consequences=consequence_unlock_door
), "source")])


staments  = [statement_is_open,statement_is_closed,statement_is_locked,statement_is_unlocked]
prerequisites = [prerequisite_close_door,prerequisite_open_door,prerequisite_lock_door,prerequisite_unlock_door]
consequences = [consequence_close_door,consequence_open_door,consequence_lock_door,consequence_unlock_door]

In [6]:
affordance_open_door = Affordance(
    name="OpenDoorAffordance",
    prerequisites=[prerequisite_open_door],
    transformations=[transformer_open_door]
)
affordance_close_door = Affordance(
    name="CloseDoorAffordance",
    prerequisites=[prerequisite_close_door],
    transformations=[transformer_close_door]
)
affordance_lock_door = Affordance(
    name="LockDoorAffordance",
    prerequisites=[prerequisite_lock_door],
    transformations=[transformer_lock_door]
)
affordance_unlock_door = Affordance(
    name="UnlockDoorAffordance",
    prerequisites=[prerequisite_unlock_door],
    transformations=[transformer_unlock_door]
)

affordances = [affordance_open_door, affordance_close_door, affordance_lock_door, affordance_unlock_door]

def print_door_status(door: Door):
    if door.is_open:
        print("The door is open")
    else:
        print("The door is closed")
    if door.is_locked:
        print("The door is locked")
    else:
        print("The door is unlocked")

In [7]:
from infinipy.goap import GOAPPlanner


In [8]:
def test_terminal_affordances():
    door = Door(
    id=str(uuid.uuid4()),
    owner_id="door_owner",  # Identifier of the door's owner or location
    name="Door",
    blocks_move=True,  # A closed door blocks movement
    blocks_los=False,  # A door doesn't generally block line of sight
    can_store=False,
    can_be_stored=False,
    can_act=False,
    can_move=False,
    can_be_moved=False,
    position=(0, 0, 0),  # Position of the door in the environment
    is_open=False,  # Custom attribute indicating if the door is open
    is_locked=True  # Custom attribute indicating if the door is locked
)
    affordances = [affordance_open_door, affordance_close_door, affordance_lock_door, affordance_unlock_door]
    planner = GOAPPlanner(affordances=affordances)
    goal = (CompositeStatement([
    (statement_is_open, "AND", "source"),
    (statement_is_closed, "AND NOT", "source"),
]), True)
    # Find terminal and partial affordances for the goal
    terminal_affordances, partial_affordances = planner.find_terminal_affordances(goal, door)

    # Check the results
    print("Terminal Affordances:", terminal_affordances)
    print("Partial Affordances:", [x[0].name for x in partial_affordances])
    print(goal[0].name, goal[1])
    plan = planner.achieve_goal(goal,door)
    print(plan)
    print(planner.print_reasoning())
    

In [9]:
def test_partial_affordances():
    # Initialize the door state
    door = Door(
    id=str(uuid.uuid4()),
    owner_id="door_owner",  # Identifier of the door's owner or location
    name="Door",
    blocks_move=True,  # A closed door blocks movement
    blocks_los=False,  # A door doesn't generally block line of sight
    can_store=False,
    can_be_stored=False,
    can_act=False,
    can_move=False,
    can_be_moved=False,
    position=(0, 0, 0),  # Position of the door in the environment
    is_open=True,  # Custom attribute indicating if the door is open
    is_locked=False  # Custom attribute indicating if the door is locked
)

    # Define a composite goal (door should be closed and locked)
    goal = (CompositeStatement([
        (statement_is_closed, "AND", "source"),
        (statement_is_locked, "AND", "source"),

    ]), True)

    # Create a planner with the defined affordances
    planner = GOAPPlanner([affordance_open_door, affordance_close_door, affordance_lock_door, affordance_unlock_door])

    # Find terminal and partial affordances for the goal
    terminal_affordances, partial_affordances = planner.find_terminal_affordances(goal, door)

    # Check the results
    print("Terminal Affordances:", terminal_affordances)
    print("Partial Affordances:", [x[0].name for x in partial_affordances])
    plan = planner.achieve_goal(goal,door)
    print([x.name for x in plan])



In [10]:
test_terminal_affordances()

Terminal Affordances: [(<infinipy.affordance.Affordance object at 0x00000189657A3490>, 2)]
Partial Affordances: []
CompositeStatement(IsOpen AND source, IsClosed AND NOT source) True
[<infinipy.affordance.Affordance object at 0x00000189657A3490>, <infinipy.affordance.Affordance object at 0x00000189657A0640>]
GOAP Planning Process Report

Time: 2023-11-30 11:01:34
Action: extract_prerequisites
Affordance: OpenDoorAffordance
Prerequisite: CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)
Composite Statement: CompositeStatement(IsClosed AND source)
--------------------------------------------------

Time: 2023-11-30 11:01:34
Action: extract_prerequisites
Affordance: OpenDoorAffordance
Prerequisite: CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)
Composite Statement: CompositeStatement(IsOpen AND NOT source)
--------------------------------------------------

Time: 20

In [11]:
test_partial_affordances()

TypeError: 'Affordance' object is not iterable

In [ ]:
planner.ongoing_plans

NameError: name 'planner' is not defined

In [ ]:
print([aff.name for aff in plan])

NameError: name 'plan' is not defined

In [ ]:
planner.print_reasoning()

GOAP Planning Process Report

Time: 2023-11-30 01:10:52
Action: extract_prerequisites
Affordance: OpenDoorAffordance
Prerequisite: CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)
Composite Statement: CompositeStatement(IsClosed AND source)
--------------------------------------------------

Time: 2023-11-30 01:10:52
Action: extract_prerequisites
Affordance: OpenDoorAffordance
Prerequisite: CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)
Composite Statement: CompositeStatement(IsOpen AND NOT source)
--------------------------------------------------

Time: 2023-11-30 01:10:52
Action: extract_prerequisites
Affordance: OpenDoorAffordance
Prerequisite: CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)
Composite Statement: CompositeStatement(IsUnlocked AND source)
-------------------------------------------

In [ ]:
#1 start with a closed and unlocked door

print_door_status(door)
# try to open the door but door is locked
print("the door can be opened",affordance_open_door.is_applicable(door))
# unlock the door
print("the door can be unlocked",affordance_unlock_door.is_applicable(door))
affordance_unlock_door.apply(door)
print_door_status(door)
# ry if the door can be opened now
print("the door can be opened",affordance_open_door.is_applicable(door))
# open the door
affordance_open_door.apply(door)
print_door_status(door)
#try to lock the door
print("the door can be locked",affordance_lock_door.is_applicable(door))
#lock the door
affordance_lock_door.apply(door)
print_door_status(door)
#try to close the door
print("the door can be closed",affordance_close_door.is_applicable(door))
#the door can be unlocked
print("the door can be unlocked",affordance_unlock_door.is_applicable(door))
#unlock the door
affordance_unlock_door.apply(door)
print_door_status(door)
#the door can be closed
print("the door can be closed",affordance_close_door.is_applicable(door))
#close the door
affordance_close_door.apply(door)
print_door_status(door)


The door is closed
The door is locked
the door can be opened False
the door can be unlocked True
The door is closed
The door is unlocked
the door can be opened True
The door is open
The door is unlocked
the door can be locked True
The door is open
The door is locked
the door can be closed False
the door can be unlocked True
The door is open
The door is unlocked
the door can be closed True
The door is closed
The door is unlocked


In [ ]:
affordance_lock_door.apply(door)
print_door_status(door)

The door is closed
The door is locked


In [ ]:
goal = (statement_is_open,door)
print("the goal is",goal[0].name)
print("the state of the goal is currently",goal[0](goal[1])[0])
#need to find which actions if applied to door will make the goal true and whether it is possible to apply them
reach_goal =[]
for affordance in affordances:
    force_true = affordance.force_consequence_true(door)
    sub_statements = force_true[0][0][0]["sub_statements"]
    for substate in sub_statements:
        if substate["statement"] == goal[0].name and substate["result"] == True:
            print(f"the action {affordance.name} will make the goal true")
            #check if the action is applicable
            if affordance.is_applicable(door):
                print(f"the action {affordance.name} is applicable")
            else:
                print(f"the action {affordance.name} is not applicable, because of the following statements: {affordance.why_not_applicable(door)}")
                reach_goal.append(affordance)
                new_goal_name = reach_goal[0].why_not_applicable(door)[0][0]
                print("setting subgoal",new_goal_name,"to",True)
                for statement in prerequisites:
                    if statement.name == new_goal_name:
                        print("found the statement object corresponding to ",statement.name)
                        new_goal = (statement,door)
                        eval_new_goal = new_goal[0](new_goal[1])
                        eval_true_new_goal = new_goal[0].force_true(new_goal[1])[0]
                        print("checking across the substatements which are not true")
                        for substatement in zip(eval_new_goal["sub_statements"],eval_true_new_goal["sub_statements"]):
                            if substatement[0]["result"] != substatement[1]["result"]:
                                print(f"the statement {substatement[0]['statement']} is  {substatement[0]['result']} , but it needs to be {substatement[1]['result']} in order to apply the action {reach_goal[0].name}")
                                second_goal_name= (substatement[0]['statement'], substatement[1]["result"])
                                print("setting subgoal",second_goal_name,"to",substatement[1]["result"])
                                for affordance in affordances:
                                    force_true = affordance.force_consequence_true(door)
                                    sub_statements = force_true[0][0][0]["sub_statements"]
                                    for substate in sub_statements:
                                        if substate["statement"] == second_goal_name[0] and substate["result"] == second_goal_name[1]:
                                            print(f"the action {affordance.name} will make the goal true")
                                            #check if the action is applicable
                                            if affordance.is_applicable(door):
                                                print(f"the action {affordance.name} is applicable")
                                            else:
                                                print(f"the action {affordance.name} is not applicable, because of the following statements: {affordance.why_not_applicable(door)}")
                                                reach_goal.append(affordance)
                
# print(force_true[0][0][0]["sub_statements"])

the goal is IsOpen
the state of the goal is currently False
the action OpenDoorAffordance will make the goal true
the action OpenDoorAffordance is not applicable, because of the following statements: [('CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)', False)]
setting subgoal CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source) to True
found the statement object corresponding to  CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)
checking across the substatements which are not true
the statement IsUnlocked is  False , but it needs to be True in order to apply the action OpenDoorAffordance
setting subgoal ('IsUnlocked', True) to True
the action UnlockDoorAffordance will make the goal true
the action UnlockDoorAffordance is applicable
the statement IsLocked is  True , but it needs to be False in order to apply

In [ ]:
from typing import List, Dict
import copy

class GOAPPlanner:
    """
    A class for Goal-Oriented Action Planning (GOAP) in a state-based system.

    Attributes:
        affordances (List[Affordance]): A list of all possible affordances (actions) in the system.
        prerequisites (Dict[str, Statement]): A mapping from statement names to Statement objects, representing prerequisites for affordances.
    """

    def __init__(self, affordances: List[Affordance]):
        """
        Initialize the GOAPPlanner with affordances and their prerequisites.

        Args:
            affordances (List[Affordance]): The list of all affordances in the system.
            prerequisites (Dict[str, Statement]): The mapping of statement names to Statement objects.
        """
        self.affordances = affordances
        self.prerequisites = self.extract_prerequisites(affordances)
        self.convert_dict={"AND": True, "AND NOT":False}
        self.planning_steps = []

    def update_planning_step(self, message: str):
        """
        Update the planning steps with a new message.

        Args:
            message (str): The message to add to the planning steps.
        """
        self.planning_steps.append(message)

    def get_and_clear_planning_steps(self) -> List[str]:
        """
        Retrieve and clear the planning steps.

        Returns:
            List[str]: The list of planning steps.
        """
        steps = self.planning_steps[:]
        self.planning_steps = []
        return steps    

    def extract_prerequisites(self, affordances: List[Affordance]) -> Dict[str, Statement]:
        """
        Extract prerequisites from a list of affordances.

        This method goes through each affordance and collects its prerequisites. It creates a dictionary
        mapping each prerequisite's name to the corresponding Statement object.

        Args:
            affordances (List[Affordance]): A list of affordances from which prerequisites are to be extracted.

        Returns:
            Dict[str, Statement]: A dictionary mapping the names of prerequisites to their corresponding Statement objects.
        """
        prerequisites = {}
        for affordance in affordances:
            for prerequisite in affordance.prerequisites:
                prerequisites[prerequisite.name] = prerequisite
                #each of htis is a composite statement so we now loop through the substatements and add all of them to the prerequisites
                for substatement in prerequisite.conditions:
                    #add the statement to the prerequisites
                    #pack the statement and condition into a composite statement
                    compstat = CompositeStatement([substatement])
                    prerequisites[compstat.name] = compstat
                    prerequisites[substatement[0].name] = substatement[0]
         
                 
        return prerequisites
    
    def get_filtered_subgoals(self,goal: Union[Tuple[CompositeStatement,bool],Tuple[Statement,bool]],entity: Door) -> List[Tuple[Statement,bool]]:
         # currently only taking the first true_dict would be worth to explore the full list in some proper examples
        goal_dict_raw= goal[0].force_true(entity)[0]
        goal_dict = copy.deepcopy(goal_dict_raw)
        
        #selects the subset of goa that are not true right now
        for substatement in goal_dict_raw["sub_statements"]:
            print(substatement["statement"],substatement["result"])
            if substatement["result"] == self.prerequisites[substatement["statement"]].apply(entity)[1]["result"]:
                #pop the substatement if it is already true
                print(f"removing the statement {substatement['statement']} from the goal because it is  {substatement['result']} and it needs to be {self.prerequisites[substatement['statement']].apply(entity)[1]['result']} in order to apply the action {goal[0].name}")
                goal_dict["sub_statements"].remove(substatement)
            else:
                print(f"the statement {substatement['statement']} is {self.prerequisites[substatement['statement']].apply(entity)[1]['result']}  , but it needs to be {substatement['result']} in order to apply the action {goal[0].name}")
        print(len(goal_dict_raw["sub_statements"]),len(goal_dict["sub_statements"]))
        return goal_dict


    def find_terminal_affordances(self, goal: Union[Tuple[CompositeStatement,bool],Tuple[Statement,bool]], entity: Door) -> List[Affordance]:
        """
        Find affordances that can directly or indirectly achieve the specified goal.

        Args:
            goal (CompositeStatement): The goal statement to be achieved as a composite statement or a statement and boolean value.
            entity: The entity over which the final goal is specified

        Returns:
            List[Affordance]: A list of affordances that can achieve the goal.
        """
        terminal_affordances = []
        partial_affordances = []
        goal_dict = self.get_filtered_subgoals(goal,entity)
        num_substatements = len(goal_dict["sub_statements"])
        #create a set of all the substatements in the goal dict
        matched_substatements_set = set()
        for affordance in self.affordances:
                num_matching_substatements = 0
                matching_substatemtents = []
                                
                # print(f"the goal {goal[0].name} is a composite statement")
                #the goals is a composite statement, same as the consequence object of the affordance so we can directly check if the goal is achieved
                #first loop is for the different composite transformers of the affordance 
                # the second loop is for the different transformers of each composite transformer
                # the third loops are the different consequences of each transformer
                # the fourth loop is for the different substatements of each consequence (should not be necessary if goal is a composite statement)
                transformations_dict_list = affordance.force_consequence_true(entity)
                for composite_transformer in transformations_dict_list:
                    for transformer in composite_transformer:
                        for consequence in transformer:
                            #tries to match the whole composite statement with the conseuquence
                            if consequence["name"] == goal[0].name and consequence["result"] == goal[1]:
                                print(consequence["name"],goal[0].name,num_substatements)
                                terminal_affordances.append((affordance,num_substatements))
                            else:
                                print("triny to match substatements")
                                #tries to match the substatements of the composite statement with the substatements of the consequence
                                for substatement in consequence["sub_statements"]:
                                    for goal_substatement in goal_dict["sub_statements"]:
                                    
                                        if substatement["statement"] == goal_substatement["statement"] and substatement["result"] == goal_substatement["result"]:
                                            num_matching_substatements += 1
                                            matching_substatemtents.append(substatement)
                                            matched_substatements_set.add(substatement["statement"])
                if num_matching_substatements == num_substatements:
                    terminal_affordances.append((affordance,num_substatements)) 
                    print("found a terminal affordance",affordance.name)
                    
                elif num_matching_substatements > 0:
                    partial_affordances.append((affordance,num_matching_substatements))
                    print("found a partial affordance",affordance.name)
                    for substatement in matching_substatemtents:
                        print(f"the follwoing substatemtens match {substatement}")
        if len(matched_substatements_set) < num_substatements and len(terminal_affordances) == 0:
            print("the goal is not achievable")
            return [],[]    
                                                                
        return terminal_affordances,partial_affordances
    


    def find_subgoals_for_affordance(self, affordance: Affordance, entity: Door) -> List[str]:
        """
        Identify subgoals that need to be achieved to make an action applicable.

        Args:
            action (Affordance): The affordance whose applicability needs to be checked.
            door (Door): The Door object representing the current state.

        Returns:
            List[str]: A list of names of subgoal statements.
        """
        subgoals = []
        why_not_applicable = affordance.why_not_applicable(entity)
        for reason in why_not_applicable:
            subgoals.append(reason[0])
        return subgoals

    def is_goal_achieved(self, goal: Union[Tuple[CompositeStatement,bool],Tuple[Statement,bool]], entity: Door) -> bool:
        """
        Check if the goal is achieved for the given door state.

        Args:
            goal (CompositeStatement): The goal to be checked.
            door (Door): The Door object representing the current state.

        Returns:
            bool: True if the goal is achieved, False otherwise.
        """
        return goal[0].apply(entity)["result"] == goal[1]

    

    def achieve_goal(self, goal: Union[Tuple[CompositeStatement, bool], Tuple[Statement, bool]], entity: Door, applied_actions: List[Affordance] = [], depth: int = 0) -> List[Affordance]:
        """
        Recursively achieve the specified goal using terminal affordances.

        Args:
            goal (Union[Tuple[CompositeStatement, bool], Tuple[Statement, bool]]): The goal or subgoal to be achieved.
            entity (Door): The entity on which the goal is being applied.
            applied_actions (List[Affordance]): Accumulator for actions applied in the process of achieving the goal.
            depth (int): Depth of recursion to prevent infinite loops.

        Returns:
            List[Affordance]: A sequence of affordances that achieve the goal.
        """
        if depth > 10:  # Avoid infinite recursion
            return applied_actions

        # Check if the goal is already achieved
        if self.is_goal_achieved(goal, entity):
            return applied_actions

        # Find terminal affordances that can achieve the goal
        terminal_affordances, _ = self.find_terminal_affordances(goal, entity)
        for affordance, _ in terminal_affordances:
            # Apply the affordance and update the entity's state
            affordance.apply(entity)
            applied_actions.append(affordance)
            
            # Check if the goal is achieved after applying the affordance
            if self.is_goal_achieved(goal, entity):
                return applied_actions
            
            # Otherwise, find the next subgoals and recurse
            next_subgoals = self.find_subgoals_for_affordance(affordance, entity)
            for subgoal_name in next_subgoals:
                subgoal_statement = self.prerequisites[subgoal_name]
                result = self.achieve_goal((subgoal_statement, True), entity, applied_actions, depth + 1)
                if self.is_goal_achieved(goal, entity):
                    return result

        return applied_actions


In [ ]:
planner = GOAPPlanner(affordances)
planner.prerequisites

{'CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)': <infinipy.statement.CompositeStatement at 0x1e2589648b0>,
 'CompositeStatement(IsClosed AND source)': <infinipy.statement.CompositeStatement at 0x1e25899df60>,
 'IsClosed': <infinipy.statement.Statement at 0x1e25835dd80>,
 'CompositeStatement(IsOpen AND NOT source)': <infinipy.statement.CompositeStatement at 0x1e25899d9f0>,
 'IsOpen': <infinipy.statement.Statement at 0x1e258428940>,
 'CompositeStatement(IsUnlocked AND source)': <infinipy.statement.CompositeStatement at 0x1e25899d930>,
 'IsUnlocked': <infinipy.statement.Statement at 0x1e25842a2c0>,
 'CompositeStatement(IsLocked AND NOT source)': <infinipy.statement.CompositeStatement at 0x1e25899d810>,
 'IsLocked': <infinipy.statement.Statement at 0x1e2587fe7d0>,
 'CompositeStatement(IsOpen AND source, IsClosed AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)': <infinipy.statement.CompositeStatement at 0x1e25

In [ ]:
print_door_status(door)

The door is closed
The door is locked


In [ ]:
goal = (CompositeStatement([
    (statement_is_open, "AND", "source"),
    (statement_is_closed, "AND NOT", "source"),
]), True)
print(goal)

(<infinipy.statement.CompositeStatement object at 0x000001E25899FD30>, True)


In [ ]:
actions = planner.achieve_goal(goal, door)

IsOpen True
the statement IsOpen is False  , but it needs to be True in order to apply the action CompositeStatement(IsOpen AND source, IsClosed AND NOT source)
IsClosed False
the statement IsClosed is True  , but it needs to be False in order to apply the action CompositeStatement(IsOpen AND source, IsClosed AND NOT source)
2 2
CompositeStatement(IsOpen AND source, IsClosed AND NOT source) CompositeStatement(IsOpen AND source, IsClosed AND NOT source) 2
triny to match substatements
triny to match substatements
triny to match substatements
IsClosed True
removing the statement IsClosed from the goal because it is  True and it needs to be True in order to apply the action CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)
IsOpen False
removing the statement IsOpen from the goal because it is  False and it needs to be False in order to apply the action CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND 

In [ ]:
print([actions.name for actions in actions])

['OpenDoorAffordance', 'UnlockDoorAffordance']


In [ ]:

terminal, partial = planner.find_terminal_affordances(goal, door)
print(terminal[0][0].name)

IsOpen True
the statement IsOpen is False  , but it needs to be True in order to apply the action CompositeStatement(IsOpen AND source, IsClosed AND NOT source)
IsClosed False
the statement IsClosed is True  , but it needs to be False in order to apply the action CompositeStatement(IsOpen AND source, IsClosed AND NOT source)
2 2
CompositeStatement(IsOpen AND source, IsClosed AND NOT source) CompositeStatement(IsOpen AND source, IsClosed AND NOT source) 2
triny to match substatements
triny to match substatements
triny to match substatements
OpenDoorAffordance


In [ ]:
goal[0].name

'CompositeStatement(IsOpen AND source, IsClosed AND NOT source)'

In [ ]:
terminal[0][0]

In [ ]:
subgoal_name = planner.find_subgoals_for_affordance(terminal[0][0], door)
subgoal = (planner.prerequisites[subgoal_name[0]], True)
print(subgoal)
print(subgoal_name)


IndexError: list index out of range

In [ ]:
terminal_subplan, partial_subplan = planner.find_terminal_affordances(subgoal, door)


NameError: name 'subgoal' is not defined

In [ ]:
partial_subplan

[]

In [ ]:
true_dict = subgoal[0].force_true(door)

In [ ]:
true_dict

[{'result': True,
  'name': 'CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)',
  'composite_string': 'The statement Door is closed is True. AND NOT The statement Door is open is False. AND The statement Door is unlocked is True. AND NOT The statement Door is locked is False.',
  'sub_statements': [{'result': True,
    'source': 'Door',
    'target': None,
    'statement': 'IsClosed',
    'stringout': 'The statement Door is closed is True.',
    'source_missing': None,
    'target_missing': None},
   {'result': False,
    'source': 'Door',
    'target': None,
    'statement': 'IsOpen',
    'stringout': 'The statement Door is open is False.',
    'source_missing': None,
    'target_missing': None},
   {'result': True,
    'source': 'Door',
    'target': None,
    'statement': 'IsUnlocked',
    'stringout': 'The statement Door is unlocked is True.',
    'source_missing': None,
    'target_missing': None},
   {'result': False,


In [ ]:
true_dict 

[{'result': True,
  'name': 'CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)',
  'composite_string': 'The statement Door is closed is True. AND NOT The statement Door is open is False. AND The statement Door is unlocked is True. AND NOT The statement Door is locked is False.',
  'sub_statements': [{'result': True,
    'source': 'Door',
    'target': None,
    'statement': 'IsClosed',
    'stringout': 'The statement Door is closed is True.',
    'source_missing': None,
    'target_missing': None},
   {'result': False,
    'source': 'Door',
    'target': None,
    'statement': 'IsOpen',
    'stringout': 'The statement Door is open is False.',
    'source_missing': None,
    'target_missing': None},
   {'result': True,
    'source': 'Door',
    'target': None,
    'statement': 'IsUnlocked',
    'stringout': 'The statement Door is unlocked is True.',
    'source_missing': None,
    'target_missing': None},
   {'result': False,


In [ ]:
terminal_subplan, partial_subplan = planner.find_terminal_affordances(subgoal, door)


IsClosed True
removing the statement IsClosed from the goal because it is  True and it needs to be True in order to apply the action CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)
IsOpen False
removing the statement IsOpen from the goal because it is  False and it needs to be False in order to apply the action CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)
IsUnlocked True
the statement IsUnlocked is False  , but it needs to be True in order to apply the action CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)
IsLocked False
the statement IsLocked is True  , but it needs to be False in order to apply the action CompositeStatement(IsClosed AND source, IsOpen AND NOT source, IsUnlocked AND source, IsLocked AND NOT source)
4 2
triny to match substatements
triny to match substatements
triny to match subst

In [ ]:
subgoal[0].conditions

[(<infinipy.statement.Statement at 0x2200daaa1d0>, 'AND', 'source'),
 (<infinipy.statement.Statement at 0x2200daaa2c0>, 'AND NOT', 'source'),
 (<infinipy.statement.Statement at 0x2200dfe5210>, 'AND', 'source'),
 (<infinipy.statement.Statement at 0x2200dfe5240>, 'AND NOT', 'source')]